# IMPORT LIBS

### IMPORT MODEL

In [ ]:
#SpeechBrainModulesGithHubPath https://github.com/speechbrain/speechbrain.git

In [ ]:
import sys
import os
notebook_path=os.path.abspath("")
sys.path.append(notebook_path+"\\"+"speechbrainrequirements")

from speechbrain.pretrained import EncoderDecoderASR
asr_model = EncoderDecoderASR.from_hparams(source="speechbrain/asr-wav2vec2-commonvoice-en", savedir="pretrained_models/asr-wav2vec2-commonvoice-en")


### IMPORT OTHER MODULES

In [2]:
import pyaudio
import wave
from playsound import playsound


from pygame import mixer
from tkinter import *
import tkinter.font as font
from tkinter import filedialog
from tkinter import StringVar

import datetime
import pytz

import glob


pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


### FUNCS

In [3]:
def save_audio(audio_name,record_seconds=10):
    
    chunk = 1024 # set the chunk size of 1024 samples
    FORMAT = pyaudio.paInt16 # sample format
    channels = 1 # mono, change to 2 if you want stereo
    sample_rate = 44100 # 44100 samples per second
    
    p = pyaudio.PyAudio()# initialize PyAudio object
    stream = p.open(format=FORMAT,           # open stream object as input & output
                    channels=channels,
                    rate=sample_rate,
                    input=True,
                    output=True,
                    frames_per_buffer=chunk)
    frames = []
    print("Recording...")
    for i in range(int(sample_rate / chunk * record_seconds)):
        current_second=int(i*chunk/sample_rate)+1
        print(current_second)
        data = stream.read(chunk)
        # if you want to hear your voice while recording stream.write(data)
        frames.append(data)
    print("Finished recording.")
    
    stream.stop_stream() # stop and close stream
    stream.close()
    p.terminate() # terminate pyaudio object
    
    # save audio file for model
    file_save_key=notebook_path+"\\"+"records"+"\\"+audio_name+".flac"
    wf = wave.open(file_save_key, "wb")# open the file in 'write bytes' mode
    wf.setnchannels(channels)# set the channels
    wf.setsampwidth(p.get_sample_size(FORMAT))# set the sample format
    wf.setframerate(sample_rate)# set the sample rate
    wf.writeframes(b"".join(frames))# write the frames as bytes
    wf.close()# close the file
    
    ## save audio file for playing
    file_save_key=notebook_path+"\\"+"records"+"\\"+audio_name+".wav"
    wf = wave.open(file_save_key, "wb")# open the file in 'write bytes' mode
    wf.setnchannels(channels)# set the channels
    wf.setsampwidth(p.get_sample_size(FORMAT))# set the sample format
    wf.setframerate(sample_rate)# set the sample rate
    wf.writeframes(b"".join(frames))# write the frames as bytes
    wf.close()# close the file
    


In [6]:

def update_btn_text(button,new_text):#update_btn_text(play_button,"text")
    button["text"]=new_text

    
def Record():
    global audio_name
    global all_flac_voice_files
    global all_wav_voice_files
    
    current_utc0_time = datetime.datetime.now()
    current_turkey_time = current_utc0_time.astimezone(pytz.timezone('Turkey')).strftime('%Y-%m-%d %H-%M-%S')
    audio_name=current_turkey_time
    
    save_audio(audio_name,record_seconds)
    
    all_flac_voice_files=glob.glob(notebook_path+"\\"+"records"+"\\*.flac")
    all_wav_voice_files=glob.glob(notebook_path+"\\"+"records"+"\\*.wav")
    
    


    
    
def Predict():
    global prediction_result
    file_read_key=notebook_path+"\\"+"records"+"\\"+audio_name+".flac"
    prediction_result=asr_model.transcribe_file(file_read_key)
    Prediction_Show()
    
    
def Previous():
    global all_wav_voice_files
    global all_flac_voice_files
    global audio_name
    
    music_index = [ i for i, x in enumerate(all_wav_voice_files) if x.split("\\")[-1].split(".")[0] == audio_name][0]
    if music_index !=0:
        print(music_index)
        audio_name=all_wav_voice_files[music_index-1].split("\\")[-1].split(".")[0]
        
def Next():
    global all_wav_voice_files
    global all_flac_voice_files
    global audio_name
    
    music_index = [ i for i, x in enumerate(all_wav_voice_files) if x.split("\\")[-1].split(".")[0] == audio_name][0]
    if music_index !=len(all_wav_voice_files)-1:
        print(audio_name)
        audio_name=all_wav_voice_files[music_index+1].split("\\")[-1].split(".")[0]
    
    
    
def Play():
    print(audio_name)
    file_read_key=notebook_path+"\\"+"records"+"\\"+audio_name+".wav"
    playsound(file_read_key)


def Increase_Time():
    global record_seconds 
    record_seconds+=1
    update_btn_text(Time_Button,str(record_seconds))
    Time_Show()

    
def Decrease_Time():
    global record_seconds
    if record_seconds>0:
        record_seconds-=1
        Time_Show()
    
def Time_Show():
    update_btn_text(Time_Button,str(record_seconds))


def Prediction_Show():
    update_btn_text(Prediction_Result_Button,prediction_result)


### PARAMETERS

In [4]:
record_seconds=5
prediction_result=""
all_flac_voice_files=glob.glob(notebook_path+"\\"+"records"+"\\*.flac")
all_wav_voice_files=glob.glob(notebook_path+"\\"+"records"+"\\*.wav")

if len(all_wav_voice_files)>0:
    audio_name=all_wav_voice_files[0].split("\\")[-1].split(".")[0]
else:
    audio_name=" "


### MAIN 

In [ ]:
#Parameters


#creating the root window 
root=Tk()
root.title('Music player App ')
#initialize mixer 
mixer.init()

#create the listbox to contain songs
#songs_list=Listbox(root,selectmode=SINGLE,bg="black",fg="white",font=('arial',15),height=12,width=100,selectbackground="gray",selectforeground="black")
#songs_list.grid(columnspan=9)

#font is defined which is to be used for the button font 
defined_font = font.Font(family='Helvetica')

#Record button
record_button=Button(root,text="Record".format(record_seconds),width =30,command=Record)
record_button['font']=defined_font
record_button.grid(row=1,column=0)

#Play button
play_button=Button(root,text="Play",width =30,command=Play)
play_button['font']=defined_font
play_button.grid(row=0,column=0)

#Predict button
predict_button=Button(root,text="Predict",width =30,command=Predict)
predict_button['font']=defined_font
predict_button.grid(row=1,column=1)

#Previous button
previous_button=Button(root,text="Previous",width =30,command=Previous)
previous_button['font']=defined_font
previous_button.grid(row=0,column=1)

#Next button
next_button=Button(root,text="Next",width =30,command=Next)
next_button['font']=defined_font
next_button.grid(row=0,column=2)

#Icrease Time button
Increase_Time_Button=Button(root,text="+(Record Time)",width =30,command=Increase_Time)
Increase_Time_Button['font']=defined_font
Increase_Time_Button.grid(row=2,column=0)

#Decrease Time button
Decrease_Time_Button=Button(root,text="-(Record Time)",width =30,command=Decrease_Time)
Decrease_Time_Button['font']=defined_font
Decrease_Time_Button.grid(row=2,column=1)

#Time Button
Time_Button=Button(root,text="".format(record_seconds),width =30,command=Time_Show)
Time_Button['font']=defined_font
Time_Button.grid(row=2,column=2)

Prediction_Result_Button=Button(root,text="",width =90,command=Prediction_Show)
Prediction_Result_Button['font']=defined_font
Prediction_Result_Button.grid(row=4,column=0,columnspan = 4)

Time_Show()

#menu 
my_menu=Menu(root)
root.config(menu=my_menu)
add_song_menu=Menu(my_menu)
my_menu.add_cascade(label="Menu",menu=add_song_menu)


mainloop()

2024-01-17 16-55-42
2024-01-17 16-55-42
2024-01-17 16-55-42
2024-01-17 17-01-30
2024-01-17 17-01-30
2024-01-20 22-57-00
2
2024-01-17 17-01-30
Recording...
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
Finished recording.
2024-01-20 23-27-58
3
2024-01-20 22-57-00
